In [3]:
import json, csv, os, time

import pandas as pd
import urllib
import tweepy

In [90]:
# Load credentials from json file
def get_credentials(file: str):
    with open(file, "r") as f:
        creds = json.load(f)
        return creds

def write_data(credentials, hashtag:str, runtime:int, language = 'en'):
    creds = get_credentials(credentials)
    auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
    auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
 
    api = tweepy.API(auth,wait_on_rate_limit=True) 
    
    tweet_file_name = f"{hashtag}_tweet_data.csv"
    user_file_name = f"{hashtag}_user_data.csv"
    
    csvFile_tweet = open(os.path.join('Data','Tweets',tweet_file_name), 'w')
    csvFile_user = open(os.path.join('Data','Users',user_file_name), 'w')
    
    csvWriter_tweet = csv.writer(csvFile_tweet)
    csvWriter_user = csv.writer(csvFile_user)
    
    start = time.time()
    for i, tweet in enumerate(tweepy.Cursor(api.search, q=f"#{hashtag}", since = '2020-08-01', lang="en").items()):
        if i != runtime*60:
            tweet_dict = dict(tweet._json)
            if (not tweet.retweeted) and ('RT @' not in tweet.text):
                csvWriter_user.writerow([tweet_dict['user']['id_str'].encode('utf-8'),
                                        tweet_dict['user']['screen_name'].encode('utf-8'),
                                        tweet_dict['user']['name'].encode('utf-8'),
                                        tweet_dict['user']['description'].encode('utf-8'),
                                        tweet_dict['user']['created_at'].encode('utf-8'),
                                        tweet_dict['user']['location'].encode('utf-8'),
                                        tweet_dict['user']['verified'],
                                        tweet_dict['user']['url'],
                                        tweet_dict['user']['listed_count'],
                                        tweet_dict['user']['favourites_count'],
                                        tweet_dict['user']['statuses_count'],
                                        tweet_dict['user']['followers_count'],
                                        tweet_dict['user']['friends_count'],
                                        tweet_dict['user']['profile_background_color'].encode('utf-8'),
                                        tweet_dict['user']['profile_use_background_image']])
                
                csvWriter_tweet.writerow([tweet_dict['user']['id_str'].encode('utf-8'),
                                        tweet_dict['text'].encode('utf-8'),
                                        tweet_dict['created_at'],
                                        tweet_dict['id_str'].encode('utf-8'),
                                        [i['text'].encode('utf-8').lower() for i in tweet_dict['entities']['hashtags']]])
        else:
            break
    end = time.time()
    print(f"Time taken to get tweets and users for {hashtag.capitalize()}: {end-start}")
    
    if os.path.exists(os.path.join('Data','Tweets',tweet_file_name)) and os.path.exists(os.path.join('Data','Users',user_file_name)):
        print(f"Tweets added successfully to {tweet_file_name} and users added successfully to {user_file_name}\n")

In [91]:
credential_path = r'Data\twitter_credentials.json'

write_data(credential_path, 'biden', 5)
write_data(credential_path, 'trump', 5)
write_data(credential_path, 'theright', 5)
write_data(credential_path, 'theleft', 5)

Time taken to get tweets and users for Biden: 9.283036470413208
Tweets added successfully to biden_tweet_data.csv and users added successfully to biden_user_data.csv

Time taken to get tweets and users for Trump: 8.195875644683838
Tweets added successfully to trump_tweet_data.csv and users added successfully to trump_user_data.csv

Time taken to get tweets and users for Theright: 0.711946964263916
Tweets added successfully to theright_tweet_data.csv and users added successfully to theright_user_data.csv

Time taken to get tweets and users for Theleft: 0.8349714279174805
Tweets added successfully to theleft_tweet_data.csv and users added successfully to theleft_user_data.csv

